In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow
from skimage.transform import resize
import cv2
from sklearn.utils import shuffle
from keras.applications.vgg16 import VGG16

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/DIP_final_project_dataset

/content/gdrive/My Drive/DIP_final_project_dataset


In [ ]:
#cloning used dataset by the papers's authors
! git clone https://github.com/yiweichen04/retina_dataset.git

Cloning into 'retina_dataset'...
remote: Enumerating objects: 617, done.
remote: Total 617 (delta 0), reused 0 (delta 0), pack-reused 617
Receiving objects: 100% (617/617), 1.67 GiB | 16.43 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (602/602), done.


In [ ]:
#directory of different sub-datasets
normal_dataset_dir = '/content/gdrive/MyDrive/DIP_final_project_dataset/retina_dataset/dataset/1_normal'
cataract_dataset_dir = '/content/gdrive/MyDrive/DIP_final_project_dataset/retina_dataset/dataset/2_cataract'
glaucoma_dataset_dir = '/content/gdrive/MyDrive/DIP_final_project_dataset/retina_dataset/dataset/2_glaucoma'
retina_disease_dataset_dir = '/content/gdrive/MyDrive/DIP_final_project_dataset/retina_dataset/dataset/3_retina_disease'

In [ ]:
#extract path to images of every sub-datasets
import os
normal_img_paths = sorted(
    [
        os.path.join(normal_dataset_dir, fname)
        for fname in os.listdir(normal_dataset_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

cataract_img_paths = sorted(
    [
        os.path.join(cataract_dataset_dir, fname)
        for fname in os.listdir(cataract_dataset_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

glaucoma_img_paths = sorted(
    [
        os.path.join(glaucoma_dataset_dir, fname)
        for fname in os.listdir(glaucoma_dataset_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

retina_disease_img_paths = sorted(
    [
        os.path.join(retina_disease_dataset_dir, fname)
        for fname in os.listdir(retina_disease_dataset_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

In [ ]:
normal_dataset = np.zeros((len(normal_img_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
cataract_dataset = np.zeros((len(cataract_img_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
glaucoma_dataset = np.zeros((len(glaucoma_img_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
retina_disease_dataset = np.zeros((len(retina_disease_img_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

In [ ]:
for i, image_id in enumerate(normal_img_paths):    
    path_image =  image_id
    image = imread(path_image)
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    normal_dataset[i] = image
print(normal_dataset.shape)

(300, 224, 224, 3)


In [ ]:
for i, image_id in enumerate(cataract_img_paths):    
    path_image =  image_id
    image = imread(path_image)
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    cataract_dataset[i] = image
print(cataract_dataset.shape)

(100, 224, 224, 3)


In [ ]:
for i, image_id in enumerate(glaucoma_img_paths):
    path_image =  image_id
    image = imread(path_image)
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    glaucoma_dataset[i] = image
print(glaucoma_dataset.shape)

(101, 224, 224, 3)


In [ ]:
for i, image_id in enumerate(retina_disease_img_paths):
    path_image =  image_id
    image = imread(path_image)
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    retina_disease_dataset[i] = image
print(retina_disease_dataset.shape)

(100, 224, 224, 3)


In [ ]:
def CLAHE(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(100, 100))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    return img

In [ ]:
#applying clahe to all images
for i, img in enumerate(normal_dataset):
    normal_dataset[i] = CLAHE(normal_dataset[i])

for i, img in enumerate(cataract_dataset):
    cataract_dataset[i] = CLAHE(cataract_dataset[i])

for i, img in enumerate(glaucoma_dataset):
    glaucoma_dataset[i] = CLAHE(glaucoma_dataset[i])

for i, img in enumerate(retina_disease_dataset):
    retina_disease_dataset[i] = CLAHE(retina_disease_dataset[i])

In [ ]:
#extracting green channels of the images
preprocessed_normal_dataset = np.zeros((len(normal_img_paths), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
preprocessed_cataract_dataset = np.zeros((len(cataract_img_paths), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
preprocessed_glaucoma_dataset = np.zeros((len(glaucoma_img_paths), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
preprocessed_retina_disease_dataset = np.zeros((len(retina_disease_img_paths), IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)

for i, img in enumerate(normal_dataset):
    preprocessed_normal_dataset[i] = img[:, :, 1]

for i, img in enumerate(cataract_dataset):
    preprocessed_cataract_dataset[i] = img[:, :, 1]

for i, img in enumerate(glaucoma_dataset):
    preprocessed_glaucoma_dataset[i] = img[:, :, 1]

for i, img in enumerate(retina_disease_dataset):
    preprocessed_retina_disease_dataset[i] = img[:, :, 1]

In [ ]:
#creating label arrays for each dataset
y_normal = np.full(len(normal_img_paths), 0)
y_cataract = np.full(len(cataract_img_paths), 1)
y_glaucoma = np.full(len(glaucoma_img_paths), 1)
y_retina_disease = np.full(len(retina_disease_img_paths), 1)

In [ ]:
#concatanating normal dataset with each of the three desease
normal_and_cataract_data = np.concatenate((normal_dataset, cataract_dataset), axis=0)
normal_and_cataract_label = np.concatenate((y_normal, y_cataract), axis=0)
normal_and_cataract_data, normal_and_cataract_label = shuffle(normal_and_cataract_data, normal_and_cataract_label)

normal_and_glaucoma_data = np.concatenate((normal_dataset, glaucoma_dataset), axis=0)
normal_and_glaucoma_label = np.concatenate((y_normal, y_glaucoma), axis=0)
normal_and_glaucoma_data, normal_and_glaucoma_label = shuffle(normal_and_glaucoma_data, normal_and_glaucoma_label)

normal_and_retina_disease_data = np.concatenate((normal_dataset, retina_disease_dataset), axis=0)
normal_and_retina_disease_label = np.concatenate((y_normal, y_retina_disease), axis=0)
normal_and_retina_disease_data, normal_and_retina_disease_label = shuffle(normal_and_retina_disease_data, normal_and_retina_disease_label)

In [ ]:
import os

import cv2
import imutils as imutils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # machine learning
from tqdm import tqdm # make your loops show a smart progress meter 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn

RANDOM_SEED = 1
IMG_SIZE = (224, 224) # size of vgg16 input

In [ ]:
vgg_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.models.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 14,739,777
Trainable params: 25,089
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split
normal_and_cataract_label = np.expand_dims(normal_and_cataract_label, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(normal_and_cataract_data, normal_and_cataract_label, test_size=0.2, random_state= 8) 

In [ ]:
EPOCHS = 25
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5
)

history = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=EPOCHS, callbacks=[early_stopping])
 
print("Training Done")
model.save("model.h5")

Epoch 1/25
18/18 [==============================] - 14s 96ms/step - loss: 3.7912 - accuracy: 0.7569 - val_loss: 0.5089 - val_accuracy: 0.8750
Epoch 2/25
18/18 [==============================] - 1s 83ms/step - loss: 2.1198 - accuracy: 0.8056 - val_loss: 0.4428 - val_accuracy: 0.9375
Epoch 3/25
18/18 [==============================] - 1s 82ms/step - loss: 1.3409 - accuracy: 0.8819 - val_loss: 0.6494 - val_accuracy: 0.9062
Epoch 4/25
18/18 [==============================] - 1s 83ms/step - loss: 1.4803 - accuracy: 0.8715 - val_loss: 0.2030 - val_accuracy: 0.9375
Epoch 5/25
18/18 [==============================] - 2s 85ms/step - loss: 0.4334 - accuracy: 0.9375 - val_loss: 0.7107 - val_accuracy: 0.9375
Epoch 6/25
18/18 [==============================] - 1s 84ms/step - loss: 0.6593 - accuracy: 0.9410 - val_loss: 0.5724 - val_accuracy: 0.9375
Epoch 7/25
18/18 [==============================] - 2s 85ms/step - loss: 0.3528 - accuracy: 0.9444 - val_loss: 0.5293 - val_accuracy: 0.9375
Training Don

In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Cataract) without augmentation:", accuracy)

Test Accuracy(Normal/Cataract) without augmentation: 0.9125


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True,
    validation_split = 0.1,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

datagen.fit(normal_and_cataract_data)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32, subset='training'),
                                 validation_data=datagen.flow(X_train, y_train, batch_size=8, subset='validation'),
                                                              epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/25
9/9 [==============================] - 6s 644ms/step - loss: 1.6115 - accuracy: 0.8299 - val_loss: 0.9343 - val_accuracy: 0.8438
Epoch 2/25
9/9 [==============================] - 5s 514ms/step - loss: 1.8990 - accuracy: 0.8264 - val_loss: 1.6199 - val_accuracy: 0.8438
Epoch 3/25
9/9 [==============================] - 5s 518ms/step - loss: 1.6316 - accuracy: 0.8542 - val_loss: 1.0728 - val_accuracy: 0.8125
Epoch 4/25
9/9 [==============================] - 5s 520ms/step - loss: 1.1733 - accuracy: 0.8472 - val_loss: 1.6450 - val_accuracy: 0.7812
Epoch 5/25
9/9 [==============================] - 5s 512ms/step - loss: 1.6466 - accuracy: 0.8333 - val_loss: 3.9295 - val_accuracy: 0.7500
Epoch 6/25
9/9 [==============================] - 5s 519ms/step - loss: 1.5061 - accuracy: 0.8542 - val_loss: 1.4945 - val_accuracy: 0.7812


In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Cataract) with augmentation", accuracy)

Test Accuracy(Normal/Cataract) with augmentation 0.7125


In [ ]:
normal_and_glaucoma_label = np.expand_dims(normal_and_glaucoma_label, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(normal_and_glaucoma_data, normal_and_glaucoma_label, test_size=0.2, random_state= 8) 

In [ ]:
EPOCHS = 25
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5
)

history = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=EPOCHS, callbacks=[early_stopping])
 
print("Training Done")

Epoch 1/25
18/18 [==============================] - 2s 87ms/step - loss: 4.8454 - accuracy: 0.6736 - val_loss: 8.3860 - val_accuracy: 0.3750
Epoch 2/25
18/18 [==============================] - 2s 86ms/step - loss: 3.6348 - accuracy: 0.7917 - val_loss: 2.5351 - val_accuracy: 0.8438
Epoch 3/25
18/18 [==============================] - 2s 86ms/step - loss: 2.1684 - accuracy: 0.8090 - val_loss: 1.2931 - val_accuracy: 0.8750
Epoch 4/25
18/18 [==============================] - 2s 86ms/step - loss: 1.2613 - accuracy: 0.9097 - val_loss: 2.7148 - val_accuracy: 0.8750
Epoch 5/25
18/18 [==============================] - 2s 86ms/step - loss: 1.0745 - accuracy: 0.8785 - val_loss: 1.3656 - val_accuracy: 0.8750
Epoch 6/25
18/18 [==============================] - 2s 88ms/step - loss: 1.2923 - accuracy: 0.8715 - val_loss: 1.6614 - val_accuracy: 0.8750
Epoch 7/25
18/18 [==============================] - 2s 86ms/step - loss: 0.9177 - accuracy: 0.9097 - val_loss: 2.5430 - val_accuracy: 0.8750
Epoch 8/25
18

In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Glaucoma) without augmentation:", accuracy)

Test Accuracy(Normal/Glaucoma) without augmentation: 0.7777777777777778


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True,
    validation_split = 0.1,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

datagen.fit(normal_and_cataract_data)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32, subset='training'),
                                 validation_data=datagen.flow(X_train, y_train, batch_size=8, subset='validation'),
                                                              epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/25
9/9 [==============================] - 5s 526ms/step - loss: 3.0195 - accuracy: 0.7431 - val_loss: 0.3442 - val_accuracy: 0.8750
Epoch 2/25
9/9 [==============================] - 5s 508ms/step - loss: 3.5541 - accuracy: 0.7083 - val_loss: 1.2761 - val_accuracy: 0.8750
Epoch 3/25
9/9 [==============================] - 5s 519ms/step - loss: 4.2194 - accuracy: 0.6944 - val_loss: 2.1631 - val_accuracy: 0.8750
Epoch 4/25
9/9 [==============================] - 5s 513ms/step - loss: 3.2770 - accuracy: 0.7257 - val_loss: 2.0781 - val_accuracy: 0.7500
Epoch 5/25
9/9 [==============================] - 5s 514ms/step - loss: 2.6821 - accuracy: 0.7917 - val_loss: 1.4672 - val_accuracy: 0.8125
Epoch 6/25
9/9 [==============================] - 5s 512ms/step - loss: 2.3235 - accuracy: 0.7743 - val_loss: 3.3648 - val_accuracy: 0.7188


In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Glaucoma) with augmentation", accuracy)

Test Accuracy(Normal/Glaucoma) with augmentation 0.7654320987654321


In [ ]:
normal_and_retina_disease_label = np.expand_dims(normal_and_retina_disease_label, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(normal_and_retina_disease_data, normal_and_retina_disease_label, test_size=0.2, random_state= 8)

In [ ]:
EPOCHS = 25
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5
)

history = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=EPOCHS, callbacks=[early_stopping])
 
print("Training Done")

Epoch 1/25
18/18 [==============================] - 2s 93ms/step - loss: 3.8124 - accuracy: 0.7604 - val_loss: 2.8212 - val_accuracy: 0.8438
Epoch 2/25
18/18 [==============================] - 2s 100ms/step - loss: 2.8318 - accuracy: 0.7812 - val_loss: 2.4403 - val_accuracy: 0.7500
Epoch 3/25
18/18 [==============================] - 2s 91ms/step - loss: 2.4537 - accuracy: 0.7986 - val_loss: 6.9333 - val_accuracy: 0.7500
Epoch 4/25
18/18 [==============================] - 2s 91ms/step - loss: 3.2664 - accuracy: 0.7917 - val_loss: 4.3218 - val_accuracy: 0.6875
Epoch 5/25
18/18 [==============================] - 2s 91ms/step - loss: 1.7070 - accuracy: 0.8576 - val_loss: 4.4069 - val_accuracy: 0.7500
Epoch 6/25
18/18 [==============================] - 2s 91ms/step - loss: 0.9732 - accuracy: 0.9062 - val_loss: 4.8819 - val_accuracy: 0.7500
Training Done


In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Retina_Disease) without augmentation:", accuracy)

Test Accuracy(Normal/Retina_Disease) without augmentation: 0.7625


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True,
    validation_split = 0.1,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

datagen.fit(normal_and_cataract_data)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32, subset='training'),
                                 validation_data=datagen.flow(X_train, y_train, batch_size=8, subset='validation'),
                                                              epochs=EPOCHS, callbacks=[early_stopping])

Epoch 1/25
9/9 [==============================] - 5s 527ms/step - loss: 4.0784 - accuracy: 0.7118 - val_loss: 3.5925 - val_accuracy: 0.7188
Epoch 2/25
9/9 [==============================] - 5s 517ms/step - loss: 4.9528 - accuracy: 0.6354 - val_loss: 2.5935 - val_accuracy: 0.8125
Epoch 3/25
9/9 [==============================] - 5s 519ms/step - loss: 4.3356 - accuracy: 0.7535 - val_loss: 4.2591 - val_accuracy: 0.5938
Epoch 4/25
9/9 [==============================] - 5s 529ms/step - loss: 5.4553 - accuracy: 0.6389 - val_loss: 1.0450 - val_accuracy: 0.8750
Epoch 5/25
9/9 [==============================] - 5s 524ms/step - loss: 4.2973 - accuracy: 0.6632 - val_loss: 1.5482 - val_accuracy: 0.8438
Epoch 6/25
9/9 [==============================] - 5s 515ms/step - loss: 4.1394 - accuracy: 0.7222 - val_loss: 2.2601 - val_accuracy: 0.8438
Epoch 7/25
9/9 [==============================] - 5s 519ms/step - loss: 3.5655 - accuracy: 0.7396 - val_loss: 1.2032 - val_accuracy: 0.8438
Epoch 8/25
9/9 [====

In [ ]:
predictions = model.predict(X_test)
predictions = [0 if x < 0.5 else 1 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy(Normal/Retina_Disease) with augmentation", accuracy)

Test Accuracy(Normal/Retina_Disease) with augmentation 0.7625


In [ ]:
#creating a multi-calss dataset
y_normal = np.full(len(normal_img_paths), 0)
y_cataract = np.full(len(cataract_img_paths), 1)
y_glaucoma = np.full(len(glaucoma_img_paths), 2)
y_retina_disease = np.full(len(retina_disease_img_paths), 3)

X = np.concatenate((normal_dataset, cataract_dataset, glaucoma_dataset, retina_disease_dataset), axis=0)
y = np.concatenate((y_normal, y_cataract, y_glaucoma, y_retina_disease), axis=0)
X, y = shuffle(X, y)

In [ ]:
y = np.expand_dims(y, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 8)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
	enc.fit(y_train.reshape(-1, 1))
	print(enc.categories_)
	y_train_enc = enc.transform(y_train.reshape(-1, 1))
	y_test_enc = enc.transform(y_test.reshape(-1, 1))
	return np.array(y_train_enc), np.array(y_test_enc)

y_train, y_test = prepare_targets(y_train, y_test)
print(y_train.shape)

[array([0, 1, 2, 3])]
(480, 4)


In [ ]:
model = tf.keras.models.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.layers[0].trainable = False

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_2 (Dropout)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 4)                 100356    
                                                                 
Total params: 14,815,044
Trainable params: 100,356
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
EPOCHS = 25
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5
)

history = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=EPOCHS, callbacks=[early_stopping])
 
print("Training Done")

Epoch 1/25
27/27 [==============================] - 3s 106ms/step - loss: 12.3039 - accuracy: 0.4329 - val_loss: 8.1571 - val_accuracy: 0.4167
Epoch 2/25
27/27 [==============================] - 3s 101ms/step - loss: 7.8967 - accuracy: 0.5903 - val_loss: 7.8162 - val_accuracy: 0.6250
Epoch 3/25
27/27 [==============================] - 3s 101ms/step - loss: 5.5941 - accuracy: 0.6736 - val_loss: 9.5250 - val_accuracy: 0.5000
Epoch 4/25
27/27 [==============================] - 3s 103ms/step - loss: 4.4728 - accuracy: 0.7130 - val_loss: 7.6908 - val_accuracy: 0.3750
Epoch 5/25
27/27 [==============================] - 3s 104ms/step - loss: 3.6123 - accuracy: 0.7731 - val_loss: 10.8478 - val_accuracy: 0.5833
Epoch 6/25
27/27 [==============================] - 3s 106ms/step - loss: 2.7332 - accuracy: 0.8102 - val_loss: 11.6080 - val_accuracy: 0.5625
Epoch 7/25
27/27 [==============================] - 3s 104ms/step - loss: 3.5070 - accuracy: 0.7801 - val_loss: 8.0299 - val_accuracy: 0.5417
Tra